### API

####  318590890 ,נתנאל פרחי

##### התקנה וייבוא ספריות
#####  קריאת הקובץ

In [2]:
!pip install googlemaps 
# בבקשה לוודא שמתקינים את הספריה הזאת בכדי שהקוד יעבוד!
import pandas as pd
import googlemaps 
# נקרא את הקובץ שמכיל את נתוני היעדים
destination=open('dests.txt')
destination=destination.read()
destination_list=destination.splitlines()


#### התחברות לשירותי גוגל 
###### API

In [9]:
#API_KEY ='' #מחקתי את המפתח בכדי שהוא לא ינוצל לרעה
try:
    gmaps = googlemaps.Client(key=API_KEY)
    distance_matrix = gmaps.distance_matrix("Tel Aviv", destination_list)
except:
    print("you have error with the request, try again")
distance_matrix

{'destination_addresses': ['İstanbul, Türkiye',
  'Amsterdam, Netherlands',
  'Valletta, Malta',
  'Basel, Switzerland',
  'Doha, Qatar'],
 'origin_addresses': ['Tel Aviv-Yafo, Israel'],
 'rows': [{'elements': [{'distance': {'text': '1,815 km', 'value': 1815227},
     'duration': {'text': '21 hours 3 mins', 'value': 75772},
     'status': 'OK'},
    {'distance': {'text': '4,533 km', 'value': 4532541},
     'duration': {'text': '2 days 0 hours', 'value': 173011},
     'status': 'OK'},
    {'distance': {'text': '3,793 km', 'value': 3792883},
     'duration': {'text': '2 days 3 hours', 'value': 183056},
     'status': 'OK'},
    {'distance': {'text': '4,093 km', 'value': 4092805},
     'duration': {'text': '1 day 20 hours', 'value': 158547},
     'status': 'OK'},
    {'distance': {'text': '2,164 km', 'value': 2164477},
     'duration': {'text': '22 hours 39 mins', 'value': 81527},
     'status': 'OK'}]}],
 'status': 'OK'}

###  שליפת מרחק , זמן וקווי רוחב ואורך לדאטה פריים

In [6]:
results=pd.DataFrame(distance_matrix["rows"][0]["elements"])
results["Target"]=destination_list
results.drop(columns="status",inplace=True)


################################################
duration_list=[]
distance_list=[]
longitude_list=[]
latitude_list=[]
#------------> איטרציות לשליפת הנתונים עבור כל מדינה
for i in range(0,len(results)):

    distance_km=round(float(results["distance"][i]['value']/1000),2)
    duration_min=results["duration"][i]["text"]
    distance_list.append(distance_km)
    duration_list.append(duration_min)
    # Geocoding location
    geocode_result = gmaps.geocode(destination_list[i])
    latitude = geocode_result[0]["geometry"]["location"]["lat"]
    longitude= geocode_result[0]["geometry"]["location"]["lng"]
    longitude_list.append(longitude)
    latitude_list.append(latitude)
#------------> השמה    
results["duration"]=duration_list
results["distance"]=distance_list
results["longitude"]=longitude_list
results["latitude"]=latitude_list
new_order=[ "Target",'distance', 'duration', 'longitude','latitude']
results=results.reindex(columns=new_order)


#### DataFrame result
###### שאלתי את ענת לגבי המרחק,והיא אמרה שנכון לשלוף את המרחק כמספר ואת הזמן כטקסט 

In [7]:
results

,Target,distance,duration,longitude,latitude
0,Istanbul,1815.23,21 hours 3 mins,28.978359,41.008238
1,Amsterdam,4532.54,2 days 0 hours,4.904139,52.367573
2,Valletta,3792.88,2 days 3 hours,14.514100,35.899237
3,Basel,4092.80,1 day 20 hours,7.588576,47.559599
4,Doha,2164.48,22 hours 39 mins,51.531040,25.285447


### שלושת הערים הכי רחוקות (בק"מ) מתל-אביב

##### מציג רק מידע רלוונטי - מרחק מתל אביב בק"מ 

In [8]:
furthest=pd.DataFrame(results.sort_values("distance",ascending=False).reset_index().iloc[:3,1:3])
furthest

,Target,distance
0,Amsterdam,4532.54
1,Basel,4092.80
2,Valletta,3792.88
